In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)

  FOLDERNAME = 'KIROV3'
  %cd /content/gdrive/My\ Drive/$FOLDERNAME
except ImportError:
  pass

Mounted at /content/gdrive
/content/gdrive/My Drive/KIROV3


In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

Parsing

In [ ]:
class Job:
    def __init__(self, data_my_job):
      self.job = data_my_job["job"]
      self.sequence = data_my_job["sequence"]
      self.release_date = data_my_job["release_date"]
      self.due_date = data_my_job["due_date"]
      self.weight = data_my_job["weight"]
    
    def set_processing_time(self, dict_tasks):
      self.total_processing_time = 0
      for task_number in self.sequence:
        task = dict_tasks[task_number]
        self.total_processing_time += task.processing_time


class Task:
    def __init__(self, data_my_task):
      self.task = data_my_task["task"]
      self.processing_time = data_my_task["processing_time"]
      self.machines = []
      self.operators = {}
      for i in range(len(data_my_task["machines"])):
        machine_number = data_my_task["machines"][i]["machine"]
        self.machines.append(machine_number)
        self.operators[machine_number] = data_my_task["machines"][i]["operators"]
    
    def get_schedule(self, release_date, final_date_previous_task, dict_machines, dict_operators, cost_machines, cost_operators):
      date_start_min = np.max([release_date, final_date_previous_task])
      possible_starting_dates = {}
      costs = {}
      for machine in self.machines:
        possible_operators = self.operators[machine]
        for operator in possible_operators:
          possible_starting_dates[(machine, operator)] = self.get_starting_date(
              date_start_min,
              dict_machines[machine].active_time,
              dict_operators[operator].active_time
              )
          costs[(machine, operator)] = possible_starting_dates[(machine, operator)] + cost_operators * dict_operators[operator].versatility + cost_machines * dict_machines[machine].versatility
      
      best_index = np.argmin(list(costs.values()))
      best_starting_date = list(possible_starting_dates.values())[best_index]
      best_machine_operator = list(possible_starting_dates.keys())[best_index]
      return best_starting_date, dict_machines[best_machine_operator[0]], dict_operators[best_machine_operator[1]]
          

    
    def get_starting_date(self, date_start_min, machine_active, operator_active):
      machine_time_found = False
      operator_time_found = False
      starting_date = date_start_min - 1
      while (not machine_time_found) or (not operator_time_found):
        starting_date += 1
        potential_time = np.arange(starting_date, starting_date + self.processing_time)

        if not any(item in machine_active for item in potential_time):
            machine_time_found = True
        
        if machine_time_found:
          if not any(item in operator_active for item in potential_time):
            operator_time_found = True
          else:
            machine_time_found = False
      return starting_date

class Machine:
  def __init__(self, id):
    self.machine = id
    self.active_time = []
    self.versatility = 0
  
class Operator:
  def __init__(self, id):
    self.operator = id
    self.active_time = []
    self.versatility = 0


def import_data(myFile):
  with open(myFile, "r") as file:
    data = json.load(file)
  
  nb_jobs = data["parameters"]["size"]["nb_jobs"]
  nb_tasks = data["parameters"]["size"]["nb_tasks"]
  nb_machines = data["parameters"]["size"]["nb_machines"]
  nb_operators = data["parameters"]["size"]["nb_operators"]

  dict_jobs = {}
  for i in range(nb_jobs):
    job_number = data['jobs'][i]["job"]
    dict_jobs[job_number] = Job(data['jobs'][i])
  
  dict_tasks = {}
  for i in range(nb_tasks):
    task_number = data['tasks'][i]["task"]
    dict_tasks[task_number] = Task(data['tasks'][i])
  
  dict_machines = {}
  for i in range(nb_machines):
    dict_machines[i+1] = Machine(i+1)
  
  dict_operators = {}
  for i in range(nb_operators):
    dict_operators[i+1] = Operator(i+1)

  ##initialize versatility of machines
  hist_machines_versatility = np.zeros(nb_machines)
  for i in dict_tasks.values():
    for m in i.machines:
      hist_machines_versatility[m-1] += 1
  
  for m in dict_machines.values():
    m.versatility = (hist_machines_versatility[m.machine - 1] - hist_machines_versatility.mean()) / hist_machines_versatility.std()

  ##initialize versatility of operators
  hist_operators_versatility = np.zeros(nb_operators)
  for i in dict_tasks.values():
    for o in np.concatenate(list(i.operators.values())):
      hist_operators_versatility[o-1] += 1

  for o in dict_operators.values():
    o.versatility = (hist_operators_versatility[o.operator - 1] - hist_operators_versatility.mean()) / hist_operators_versatility.std()
  
  return dict_jobs, dict_tasks, dict_machines, dict_operators, data["parameters"]["costs"]

In [ ]:
def compute_cost(dict_jobs, dict_tasks, dict_costs, Schedule):
  cost = 0
  for job in list(dict_jobs.values()):
    last_task = job.sequence[-1]
    last_task_start = Schedule[Schedule["task"] == last_task]["start"].iloc[0]
    last_task_end = last_task_start + dict_tasks[last_task].processing_time
    cost += job.weight * last_task_end
    cost += job.weight * np.max([last_task_end - job.due_date, 0]) * dict_costs['tardiness']
    if last_task_end > job.due_date:
      cost += job.weight * dict_costs['unit_penalty']
  return cost

Heuristic

In [ ]:
import random

In [ ]:
def PileVersatilityHeuristic(dict_jobs, dict_tasks, dict_machines, dict_operators, dict_costs, alpha, beta): 

  list_jobs_sorted = sorted(list(dict_jobs.values()), key=lambda x: x.weight, reverse=True)

  #for idx_j, j in enumerate(list_jobs_sorted):
   # for idx_i, i in enumerate(list_jobs_sorted):
    #  if idx_i > idx_j: continue;
     # if i.due_date == j.due_date and j.weight > i.weight:
      #  list_jobs_sorted[idx_i], list_jobs_sorted[idx_j] = list_jobs_sorted[idx_j], list_jobs_sorted[idx_i]


  Schedule = pd.DataFrame(columns=['task', 'start', 'machine', 'operator'])

  for idx_job, j in enumerate(list_jobs_sorted):

    for idx_task, task_nb in enumerate(j.sequence):
      i = dict_tasks[task_nb]
      if idx_task == 0:
        end_date_previous_task_of_same_job = 0
      

      starting_date, first_available_machine, first_available_operator = i.get_schedule(j.release_date, end_date_previous_task_of_same_job, dict_machines, dict_operators, cost_machines = alpha, cost_operators = beta)

      first_available_machine.active_time += list(np.arange(starting_date, starting_date + i.processing_time))
      first_available_operator.active_time += list(np.arange(starting_date, starting_date + i.processing_time))

      Schedule.loc[len(Schedule.index)] = [i.task, starting_date, first_available_machine.machine, first_available_operator.operator] 


      end_date_previous_task_of_same_job = starting_date + i.processing_time + 1
    
  score = (compute_cost(dict_jobs, dict_tasks, dict_costs, Schedule))

  return Schedule, score

In [ ]:
Solutions = None
Final = None
Alphas = [0] #np.logspace(-5, 0, 5)
Betas = [0] #np.logspace(-5, 0, 5)
score = []
for name_file in ['huge.json']:
  for alpha in Alphas:
    for beta in Betas:
      dict_jobs, dict_tasks, dict_machines, dict_operators, dict_costs = import_data(name_file)

      Solutions = PileVersatilityHeuristic(dict_jobs, dict_tasks, dict_machines, dict_operators, dict_costs, alpha, beta)
      score.append(Solutions[1])
      
      if score[-1] == np.min(score):
        Final = Solutions[0]



In [ ]:
score

[62468]

In [ ]:
WriteSolution('sol_huge.json', Final)

In [ ]:
def WriteSolution(name_file, Schedule):
  with open('solutions/' + 'sol_' + name_file, 'w') as file:
    sol = Schedule.to_json(orient='records', indent=2)
    file.write(sol)

for name_file, Schedule in Solutions:
  WriteSolution(name_file, Schedule)

In [ ]:
Solutions[0][1].head(30)